# Information Retrieval Lab WiSe 2024/2025: Baseline Retrieval System

This Jupyter notebook serves as a baseline retrieval system that you can improve upon.
We use subsets of the MS MARCO datasets to retrieve passages of web documents.
We will show you how to create a software submission to TIRA from this notebook.

An overview of all corpora that we use in the current course is available at [https://tira.io/datasets?query=ir-lab-wise-2024](https://tira.io/datasets?query=ir-lab-wise-2024). The dataset IDs for loading the datasets are:

- `ir-lab-wise-2024/subsampled-ms-marco-deep-learning-20241201-training`: A subsample of the TREC 2019/2020 Deep Learning tracks on the MS MARCO v1 passage dataset. Use this dataset to tune your system(s).
- `ir-lab-wise-2024/subsampled-ms-marco-rag-20241202-training` (_work in progress_): A subsample of the TREC 2024 Retrieval-Augmented Generation track on the MS MARCO v2.1 passage dataset. Use this dataset to tune your system(s).
- `ir-lab-wise-2024/ms-marco-rag-20241203-test` (work in progress): The test corpus that we have created together in the course, based on the MS MARCO v2.1 passage dataset. We will use this dataset as the test dataset, i.e., evaluation scores become available only after the submission deadline.

### Step 1: Import libraries

We will use [tira](https://tira.io/), an information retrieval shared task platform, and [ir_dataset](https://ir-datasets.com/) for loading the datasets. Subsequently, we will build a retrieval system with [PyTerrier](https://github.com/terrier-org/pyterrier), an open-source search engine framework.

First, we need to install the required libraries.

In [1]:
!pip3 install 'tira>=0.0.139' ir-datasets 'python-terrier==0.10.0'

In [2]:
!wget https://files.webis.de/software/pyterrier-plugins/custom-terrier-token-processing-1.0-SNAPSHOT-jar-with-dependencies.jar -O /root/.pyterrier/custom-terrier-token-processing-0.0.1.jar

--2024-12-08 23:18:09--  https://files.webis.de/software/pyterrier-plugins/custom-terrier-token-processing-1.0-SNAPSHOT-jar-with-dependencies.jar
Resolving files.webis.de (files.webis.de)... 141.54.132.200
Connecting to files.webis.de (files.webis.de)|141.54.132.200|:443... 

connected.
HTTP request sent, awaiting response... 200 OK
Length: 499865236 (477M) [application/java-archive]
Saving to: ‘/root/.pyterrier/custom-terrier-token-processing-0.0.1.jar’

/root/.pyterrier/cu 100%[===================>] 476.71M  93.1MB/s    in 5.2s    

2024-12-08 23:18:23 (92.0 MB/s) - ‘/root/.pyterrier/custom-terrier-token-processing-0.0.1.jar’ saved [499865236/499865236]



In [3]:
import pyterrier as pt
import pandas as pd
pd.set_option('display.max_colwidth', 0)

if not pt.started():
    pt.init(boot_packages=['mam10eks:custom-terrier-token-processing:0.0.1'])
    from jnius import autoclass

PyTerrier 0.10.0 has loaded Terrier 5.10 (built by craigm on 2024-08-22 17:33) and terrier-helper 0.0.8



Create an API client to interact with the TIRA platform (e.g., to load datasets and submit runs).

In [4]:
from tira.third_party_integrations import ensure_pyterrier_is_loaded
from tira.rest_api_client import Client

ensure_pyterrier_is_loaded()
tira = Client()

### Step 2: Load the dataset

We load the dataset by its ir_datasets ID (as listed in the Readme). Just be sure to add the `irds:` prefix before the dataset ID to tell PyTerrier to load the data from ir_datasets.

In [5]:
from pyterrier import get_dataset

pt_dataset = get_dataset('irds:ir-lab-wise-2024/subsampled-ms-marco-deep-learning-20241201-training')

### Step 3: Build an index

We will then create an index from the documents in the dataset we just loaded.

In [6]:
from pyterrier import IterDictIndexer

indexer_porter = IterDictIndexer(
    # Store the index in the `index` directory.
    "../data/index",
    meta={'docno': 50, 'text': 4096},
    # If an index already exists there, then overwrite it.
    overwrite=True,
    stemmer='PorterStemmer'
)
index_porter = indexer_porter.index(pt_dataset.get_corpus_iter())


ir-lab-wise-2024/subsampled-ms-marco-deep-learning-20241201-training documents:  38%|███▊      | 26034/68261 [00:08<00:08, 4787.89it/s]

23:18:48.285 [ForkJoinPool-1-worker-3] WARN org.terrier.structures.indexing.Indexer -- Adding an empty document to the index (6114613) - further warnings are suppressed


ir-lab-wise-2024/subsampled-ms-marco-deep-learning-20241201-training documents: 100%|██████████| 68261/68261 [00:13<00:00, 4976.49it/s] 


23:18:55.533 [ForkJoinPool-1-worker-3] WARN org.terrier.structures.indexing.Indexer -- Indexed 1 empty documents


In [7]:
indexer_none = IterDictIndexer(
    "../data/index_none",
    meta={'docno': 50, 'text': 4096},
    overwrite=True,
    stemmer = None
)

index_none= indexer_none.index(pt_dataset.get_corpus_iter())

ir-lab-wise-2024/subsampled-ms-marco-deep-learning-20241201-training documents:  38%|███▊      | 26119/68261 [00:03<00:04, 8966.41it/s] 

23:19:00.804 [ForkJoinPool-2-worker-3] WARN org.terrier.structures.indexing.Indexer -- Adding an empty document to the index (6114613) - further warnings are suppressed


ir-lab-wise-2024/subsampled-ms-marco-deep-learning-20241201-training documents: 100%|██████████| 68261/68261 [00:08<00:00, 8338.18it/s] 


23:19:07.595 [ForkJoinPool-2-worker-3] WARN org.terrier.structures.indexing.Indexer -- Indexed 1 empty documents


In [8]:
indexer_Snowball = IterDictIndexer(
    "../data/index_Snowball",
    meta={'docno': 50, 'text': 4096},
    overwrite=True,
    stemmer = 'EnglishSnowballStemmer'
)

index_Snowball = indexer_Snowball.index(pt_dataset.get_corpus_iter())

ir-lab-wise-2024/subsampled-ms-marco-deep-learning-20241201-training documents:  37%|███▋      | 25539/68261 [00:03<00:05, 8476.06it/s]

23:19:18.805 [ForkJoinPool-3-worker-3] WARN org.terrier.structures.indexing.Indexer -- Adding an empty document to the index (6114613) - further warnings are suppressed


ir-lab-wise-2024/subsampled-ms-marco-deep-learning-20241201-training documents: 100%|██████████| 68261/68261 [00:08<00:00, 8383.62it/s]


23:19:24.795 [ForkJoinPool-3-worker-3] WARN org.terrier.structures.indexing.Indexer -- Indexed 1 empty documents


In [9]:
indexer_LemurKrovetz = IterDictIndexer(
    "../data/index_LemurKrovetz",
    meta={'docno': 50, 'text': 4096},
    overwrite=True,
    stemmer = 'LemurKrovetzStemmer'
)

index_LemurKrovetz = indexer_LemurKrovetz.index(pt_dataset.get_corpus_iter())

ir-lab-wise-2024/subsampled-ms-marco-deep-learning-20241201-training documents:  38%|███▊      | 25935/68261 [00:04<00:05, 7362.69it/s]

23:19:44.467 [ForkJoinPool-4-worker-3] WARN org.terrier.structures.indexing.Indexer -- Adding an empty document to the index (6114613) - further warnings are suppressed


ir-lab-wise-2024/subsampled-ms-marco-deep-learning-20241201-training documents: 100%|██████████| 68261/68261 [00:10<00:00, 6582.59it/s] 


23:19:51.480 [ForkJoinPool-4-worker-3] WARN org.terrier.structures.indexing.Indexer -- Indexed 1 empty documents


In [10]:
indexer_standfordLemmatizer = IterDictIndexer(
    "../data/index_standfordLemmatizer",
    meta={'docno': 50, 'text': 4096},
    overwrite=True,
    stemmer = 'StanfordLemmatizer'
)

index_standfordLemmatizer = indexer_standfordLemmatizer.index(pt_dataset.get_corpus_iter())

ir-lab-wise-2024/subsampled-ms-marco-deep-learning-20241201-training documents:  38%|███▊      | 26113/68261 [00:34<00:51, 816.60it/s]

23:20:38.224 [ForkJoinPool-5-worker-3] WARN org.terrier.structures.indexing.Indexer -- Adding an empty document to the index (6114613) - further warnings are suppressed


ir-lab-wise-2024/subsampled-ms-marco-deep-learning-20241201-training documents: 100%|██████████| 68261/68261 [01:23<00:00, 821.88it/s]


23:21:28.590 [ForkJoinPool-5-worker-3] WARN org.terrier.structures.indexing.Indexer -- Indexed 1 empty documents


### Step 4: Define the retrieval pipeline

We will define a simple retrieval pipeline using just BM25 as a baseline. For details, refer to the PyTerrier [documentation](https://pyterrier.readthedocs.io) or [tutorial](https://github.com/terrier-org/ecir2021tutorial).

In [11]:
from pyterrier import BatchRetrieve

bm25_porter = BatchRetrieve(index_porter, wmodel="BM25")
bm25_none = BatchRetrieve(index_none, wmodel="BM25")
bm25_snowball = BatchRetrieve(indexer_Snowball, wmodel="BM25")
bm25_lemurkrovetz = BatchRetrieve(indexer_LemurKrovetz, wmodel="BM25")
bm25_standfordLemmatizer = BatchRetrieve(index_standfordLemmatizer, wmodel="BM25")

### Step 5: Create the run
In the next steps, we would like to apply our retrieval system to some topics, to prepare a 'run' file, containing the retrieved documents.

First, let's have a short look at the first three topics:

In [12]:
# The `'text'` argument below selects the topics `text` field as the query.
pt_dataset.get_topics('text').head(3)

,qid,query
0,1030303,who is aziz hashim
1,1037496,who is rep scalise
2,1043135,who killed nicholas ii of russia


Now, retrieve results for all the topics (may take a while):

In [14]:
run = bm25_lemurkrovetz(pt_dataset.get_topics('text'))

That's it for the retrieval. Here are the first 10 entries of the run:

In [15]:
run.head(10)

,qid,docid,docno,rank,score,query
0,1030303,53852,8726436,0,31.681671,who is aziz hashim
1,1030303,56041,8726433,1,25.966276,who is aziz hashim
2,1030303,62116,8726435,2,23.863442,who is aziz hashim
3,1030303,32183,8726429,3,23.391821,who is aziz hashim
4,1030303,35867,8726437,4,21.030669,who is aziz hashim
5,1030303,17637,8726430,5,19.967200,who is aziz hashim
6,1030303,42957,7156982,6,19.967200,who is aziz hashim
7,1030303,21803,8726434,7,19.474804,who is aziz hashim
8,1030303,59828,1305520,8,17.849161,who is aziz hashim
9,1030303,60002,3302257,9,17.832781,who is aziz hashim


### Step 6: Persist and upload run to TIRA

The output of our retrieval system is a run file. This run file can later (and, e.g., in a different notebook or by a different person) be statistically evaluated. We will therefore first upload the run to TIRA.

In [16]:
from tira.third_party_integrations import persist_and_normalize_run

persist_and_normalize_run(
    run,
    # Give your approach a short but descriptive name tag.
    system_name='bm25-modifiedStemmer', 
    default_output='../data/runs',
    upload_to_tira=pt_dataset,
)

The run file is normalized outside the TIRA sandbox, I will store it at "../data/runs".
Done. run file is stored under "../data/runs/run.txt.gz".
Run uploaded to TIRA. Claim ownership via: https://www.tira.io/claim-submission/f3476d3b-6066-434b-8df1-1bc77b9fdfd7


Click on the link in the cell output above to claim your submission on TIRA.

# Step 7: Improve

Building your own index can be already one way that you can try to improve upon this baseline (if you want to focus on creating good document representations). Other ways could include reformulating queries or tuning parameters or building better retrieval pipelines.

In [13]:
from pyterrier import Experiment 

Experiment(
    [bm25_porter, bm25_none, bm25_snowball, bm25_lemurkrovetz, bm25_standfordLemmatizer],
    topics = pt_dataset.get_topics("text"),
    qrels = pt_dataset.get_qrels(),
    eval_metrics=["ndcg_cut_10"],
    names=["BM25 with Porter Stemmer", "BM25 with no Stemmer", "BM25 with Snowball Stemmer", "BM25 with LemurKrovetz Stemmer", "BM25 with Standford Lemmatizer"]
)

,name,ndcg_cut_10
0,BM25 with Porter Stemmer,0.489469
1,BM25 with no Stemmer,0.468890
2,BM25 with Snowball Stemmer,0.489216
3,BM25 with LemurKrovetz Stemmer,0.490341
4,BM25 with Standford Lemmatizer,0.482512


In [10]:
pt_dataset.get_qrels()

,qid,docno,label,iteration
0,19335,1082489,0,0
1,19335,109063,0,0
2,19335,1231806,0,0
3,19335,1324075,0,0
4,19335,1509459,0,0
...,...,...,...,...
12971,1136962,8336581,0,0
12972,1136962,8380913,0,0
12973,1136962,8537921,0,0
12974,1136962,937258,1,0
